# Proximal Algorithms - L1 Regularization

# Algorithm explanation

1. 采用 L1 正则的求逻辑回归的参数梯度

$$\underset{\theta}{\text{min}} \sum_i^N(y_i-\sigma(x_i^T\theta))^2 + \lambda||\theta||_1$$


2. 简略算法原理

    1. proxmial operator 定义
        
        $$\text{prox}_f(v) = \underset{x}{\text{argmin}}(f(x)+\frac{1}{2}||x-v||^2_2)$$
        
    2. 对于 proxmial operator 有一个良好的性质
        
        $$ x^* = \underset{x}{\text{argmin}} f(x) \text{    当且仅当    } x^* = prox_f(x^*)$$
        
    3. 对于一个函数 $z$ 由一个可微函数 $f$ 和一个不可微分函数 $g$ 组成，对于 $z$ 有如下定义重要公式
        
        $$z(x) = f(x) + g(x)$$
        
        $$x^{k+1} := \text{prox}_{\lambda^kg}(x^k - \lambda^k \nabla f(x^k))$$
        
    4. 应用在参数L1参数求解当中 $prox_f(x)$ 中的 $x$ 就是我门需要求的 $\theta$， $\lambda^k$ 是学习步长
        
        $$w:=\theta^k-\lambda^k\nabla f_{\theta^k}(X)$$
        
        $$\theta^{k+1} = \text{prox}_{{\lambda^k}g}(w)$$
        
    5. 求解上面式子的结果，$\lambda$ 是 L1 正则的惩罚系数，$\lambda^k$ 是学习步长，$\theta$ 是一个向量
    
        $$[\theta^{k+1}]_i=\left\{\begin{aligned}
        w_i - \lambda\lambda^k, & \text{ if } & w_i > \lambda\lambda^k\\
        0, & \text{ if } & \lambda\lambda^k < w_i < -\lambda\lambda^k\\
        w_i + \lambda\lambda^k, & \text{ if } & w_i < -\lambda\lambda^k\\
                                  \end{aligned}\right.$$
                                                    




3. 详细证明过程附件中

# Test Data

In [39]:
import jax
import jax.numpy as jnp
import jax.nn as jnn
from jax import grad, vmap

In [32]:
key = jax.random.PRNGKey(0)

x = jax.random.normal(key, (1000,5))
true_theta = jnp.array([2.0, 2.0, 0.0, 0.0, 0.0])
theta = jax.random.normal(key, (5, ))
y = (jnn.sigmoid(jnp.dot(x, true_theta)) >= 0.5).astype(jnp.float32)

print('generated theta', true_theta)
print('initial theta', theta)

generated theta [2. 2. 0. 0. 0.]
initial theta [ 0.18784378 -1.2833427  -0.27109176  1.2490592   0.24446994]


# Code Implementation

## Normal Version

In [33]:
def predict(theta, x):
    return jnn.sigmoid(jnp.dot(x, theta))

def get_lr_grad(theta, x, y_true):
    y_pre = predict(theta, x)
    return jnp.dot(x.T, y_pre - y_true) / x.shape[0]

def get_lr_loss(theta, x, y_true):
    y_pre = predict(theta, x)
    return -jnp.mean(y_true * jnp.log(y_pre) + (1 - y_true) * jnp.log(1 - y_pre))


def soft_threshold(vec_theta, threshold):
    new_vec_theta = []

    for theta in vec_theta:
        if theta > threshold:
            new_vec_theta.append(theta - threshold)
        elif theta < -threshold:
            new_vec_theta.append(theta + threshold)
        else:
            new_vec_theta.append(0)
    
    return jnp.array(new_vec_theta)
    

def proximal_method_update(theta, x, y_true, lr, penalty):
    w = theta - lr * get_lr_grad(theta, x, y_true)
    new_theta = soft_threshold(w, lr * penalty)

    return jnp.array(new_theta)


def model_train(theta, x, y_true, lr, penalty, esp, max_iter):
    current_iter, current_esp = 1, 1
    
    while max_iter > current_iter and current_esp > esp:
        
        loss = get_lr_loss(theta, x, y_true) + penalty * jnp.linalg.norm(theta, 1)
        new_theta = proximal_method_update(theta, x, y_true, lr, penalty)
        current_esp = jnp.mean(jnp.abs(new_theta - theta))
        
        theta = new_theta
        current_iter += 1
        
#         print('{}  loss: {:.4f} theta: {}'.format(current_iter-1, loss, theta))

    return new_theta

In [34]:
lr = 0.05
penalty = 0.1
esp = 1e-4
max_iter = 3000

proximal_method_theta = model_train(theta, x, y, lr, penalty, esp, max_iter)
proximal_method_theta

DeviceArray([0.99816716, 0.91049355, 0.        , 0.        , 0.        ],            dtype=float32)

## Jax Version

In [40]:
@grad
def jax_lr_loss(theta, x, y):
    y_pre = jnn.sigmoid(jnp.dot(x, theta))
    return -(jnp.mean(y*jnp.log(y_pre) + (1-y)*jnp.log(1-y_pre)))

In [41]:
jax_lr_loss(theta, x, y)

DeviceArray([-0.24260923, -0.4717243 , -0.08309114,  0.1896846 ,
              0.04510983], dtype=float32)

In [42]:
get_lr_grad(theta,x, y)

DeviceArray([-0.24260938, -0.47172442, -0.08309112,  0.1896846 ,
              0.04510981], dtype=float32)